In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [4]:
import os
from tqdm.autonotebook import tqdm, trange

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time

/tmp/ipykernel_26044/3508907965.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


ModuleNotFoundError: No module named 'matplotlib'

In [12]:
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used.")
    device = torch.device('cuda')
else:
    print("CUDA is not available. CPU will be used.")
    device = torch.device('cpu')

CUDA is not available. CPU will be used.


/root/.pyenv/versions/3.11.5/lib/python3.11/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 802: system not yet initialized (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [7]:
# Преобразование обучающих данных для расширения обучающей выборки и её нормализация
# Для валидационной (тестовой) выборки только нормализация
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(244),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(244),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}



full_dataset = datasets.ImageFolder(root='train/simpsons_dataset')
total_size = len(full_dataset)
val_size = int(0.2 * total_size)  # let's assume 20% for validation
train_size = total_size - val_size

# Split the dataset into train and validation
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']



# специальный класс для загрузки данных в виде батчей
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=2)
               for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

NameError: name 'image_datasets' is not defined

In [5]:
! nvidia-smi


Mon Jul 22 15:33:26 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.06              Driver Version: 555.42.06      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          Off |   00000000:07:00.0 Off |                    0 |
| N/A   25C    P0             52W /  400W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [6]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

CudaSupportError: Error at driver init: 
Call to cuInit results in CUDA_ERROR_SYSTEM_NOT_READY (802):

In [1]:
import torch

# Custom BCE Loss Implementation
def custom_bce_loss(y_real, y_pred):
    return (y_pred - y_real * y_pred + torch.log(1 + torch.exp(-y_pred))).mean()

# Test data
y_real = torch.tensor([1, 0, 1, 0], dtype=torch.float32)
y_pred = torch.tensor([3.0, -2.0, 1.0, 0.0], dtype=torch.float32)  # Logits

# Calculate custom BCE Loss
custom_loss = custom_bce_loss(y_real, y_pred)
print("Custom BCE Loss:", custom_loss.item())

# PyTorch built-in BCEWithLogitsLoss
bce_loss = torch.nn.BCEWithLogitsLoss()
torch_loss = bce_loss(y_pred, y_real)
print("Torch BCEWithLogitsLoss:", torch_loss.item())


Custom BCE Loss: 0.2954810559749603
Torch BCEWithLogitsLoss: 0.2954810857772827
